In [ ]:
!pip install ndvi2gif

In [2]:
import geemap
import ee
from ndvi2gif import NdviSeasonality

In [3]:
ee.Authenticate()
ee.Initialize(project='cursocsic2025')

In [4]:
# Test rápido
from ndvi2gif import NdviSeasonality

# Test S3
s2_test = NdviSeasonality(sat='S2', index='ndvi')
print(s2_test.get_available_indices('S2'))

There we go again...
Applying cloud filter to Sentinel-2: max 20% cloud cover
Using MODIS Terra + Aqua LST (maximum coverage)
Using all Sentinel-1 orbits (ascending + descending).
Applying S1 ARD preprocessing:
  - Speckle filter: REFINED_LEE
  - Terrain correction: True
  - Terrain model: VOLUME
Sentinel-2 collection configured with cloud filtering
['avi', 'awei', 'aweinsh', 'cire', 'cri1', 'cri2', 'evi', 'gndvi', 'ireci', 'lai', 'lst', 'mcari', 'mndwi', 'msi', 'mtci', 'nbr', 'nbri', 'ndbi', 'ndci', 'ndmi', 'ndre', 'ndsi', 'ndti', 'ndvi', 'ndwi', 'nmi', 'pri', 'psri', 'reip', 's2rep', 'savi', 'utfvi', 'vci', 'wdrvi', 'wi2015']


In [5]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [15]:
roi = Map.draw_last_feature

## Uso Básico

In [16]:
# Doñana NDVI Stats
myclass = NdviSeasonality(roi=roi, sat='S2',
                          key='max',
                          #percentile=75,
                          periods=12,
                          start_year=2020,
                          end_year=2023,
                          index='ndvi',
                          cloud_filter=False)

Sentinel2 = myclass.get_year_composite()
vizParams = {'bands': ['january', 'april', 'august'], 'min': 0.1, 'max':0.8}

Map.addLayer(Sentinel2, vizParams, 'MedianCMax2')

There we go again...
Applying cloud filter to Sentinel-2: max 50% cloud cover
Using MODIS Terra + Aqua LST (maximum coverage)
Using all Sentinel-1 orbits (ascending + descending).
Applying S1 ARD preprocessing:
  - Speckle filter: REFINED_LEE
  - Terrain correction: True
  - Terrain model: VOLUME
Sentinel-2 collection configured with cloud filtering
Year 2020: Successfully processed 12 periods using ndvi index
Year 2021: Successfully processed 12 periods using ndvi index
Year 2022: Successfully processed 12 periods using ndvi index


### Geometrías Aceptadas

In [17]:
roi_shapefile = 'path/to/your/shapefile.shp'
roi_geojson = 'path/to/your/file.geojson'

# Maps Geometry
roi_draw = Map.draw_last_feature

# ee Geometry
roi_coords = ee.Geometry.Rectangle([-6.766, 36.776, -5.867, 37.202])  # [xMin, yMin, xMax, yMax]

# Landsat Path/Rows
roi_landsat = 'wrs:202,034'  # Format: 'wrs:path,row'

# Sentinel 2 Tiles
roi_s2_tile = 's2:T30TYN'  # Format: 's2:TILE_ID'

# Use European Long-Term Ecosystem Research sites (eLTER sites)
roi_elter = 'deimsid:ab8278e6-0b71-4b36-a6d2-e8f34aa3df30'  # Format: 'deimsid:SITE_ID'
# Note: it could required 'deims' package: pip install deims in case conda couldn't make the installation

In [18]:
Map2 = geemap.Map()
Map2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [19]:
deims = 'deimsid:https://deims.org/bcbc866c-3f4f-47a8-bbbc-0a93df6de7b2'

# Doñana NDVI Stats
myclass = NdviSeasonality(roi=deims, sat='S2',
                          key='percentile',
                          percentile=75,
                          periods=12,
                          start_year=2020,
                          end_year=2023,
                          index='ndvi',
                          cloud_filter=False)

S2 = myclass.get_year_composite()
vizParams = {'bands': ['february', 'may', 'september'], 'min': 0.1, 'max':0.8}

Map2.addLayer(S2.median(), vizParams, 'Median')

There we go again...
Con Deims hemos topado, amigo Sancho...
Using MODIS Terra + Aqua LST (maximum coverage)
Using all Sentinel-1 orbits (ascending + descending).
Applying S1 ARD preprocessing:
  - Speckle filter: REFINED_LEE
  - Terrain correction: True
  - Terrain model: VOLUME
Year 2020: Successfully processed 12 periods using ndvi index
Year 2021: Successfully processed 12 periods using ndvi index
Year 2022: Successfully processed 12 periods using ndvi index


### Como vemos, con esta herramienta es muy fácil y rápido obtener productos que "sinteticen" un período en base a una variable. Como por ejemplo lo visto en el curso sobre el MNDWI

In [24]:
Map3 = geemap.Map()
Map3

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Mediana MNDWI de toda la serie Landsat (L4-5-7-8-9 fusionados)
roi_landsat = 'wrs:202,034'
proc = NdviSeasonality(
    roi=roi_landsat,
    sat='Landsat',
    index='mndwi',
    key='max',
    #percentile=95,
    periods=1,
    start_year=1984,
    end_year=2025,
    cloud_filter=True,
    max_cloud_cover=10
)

# 1 imagen por año (banda 'p1'); después hacemos la mediana interanual
coll = proc.get_year_composite()
mndwi_alltime_max = coll.median()

# Visualización (rango típico MNDWI ~[-1, 1]; agua suele ser positivo)
viz = {'min': 0.2, 'max': 0.6}
Map3.addLayer(mndwi_alltime_max, viz, 'Landsat MNDWI Max (1998-1990)')

In [29]:
Map4 = geemap.Map()
Map4

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### **Ejercicio: Calcular la inundación máxima de invierno de los años 1988 a 1992 y 2008 a 2012**

In [33]:
# Start coding

## Exportación de imagenes

In [34]:
Map5 = geemap.Map()
Map5

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [35]:
roi = Map5.draw_last_feature

In [36]:
# Doñana NDVI Stats
myclass = NdviSeasonality(roi=roi, sat='S2',
                          key='max',
                          #percentile=75,
                          periods=12,
                          start_year=2020,
                          end_year=2023,
                          index='ndvi',
                          cloud_filter=False)

Sentinel2 = myclass.get_year_composite()
vizParams = {'bands': ['january', 'april', 'august'], 'min': 0.1, 'max':0.8}

Map5.addLayer(Sentinel2, vizParams, 'MedianCMax2')

There we go again...
Using MODIS Terra + Aqua LST (maximum coverage)
Using all Sentinel-1 orbits (ascending + descending).
Applying S1 ARD preprocessing:
  - Speckle filter: REFINED_LEE
  - Terrain correction: True
  - Terrain model: VOLUME
Year 2020: Successfully processed 12 periods using ndvi index
Year 2021: Successfully processed 12 periods using ndvi index
Year 2022: Successfully processed 12 periods using ndvi index


In [37]:
myclass.get_export(
    scale=30,
    crs="EPSG:32629"
)

Year 2020: Successfully processed 12 periods using ndvi index
Year 2021: Successfully processed 12 periods using ndvi index
Year 2022: Successfully processed 12 periods using ndvi index
3
Exporting /content/S2_ndvi_max_2020.tif 

Generating URL ...
Please wait ...
Data downloaded to /content/S2_ndvi_max_2020.tif
Exporting /content/S2_ndvi_max_2021.tif 

Generating URL ...
Please wait ...
Data downloaded to /content/S2_ndvi_max_2021.tif
Exporting /content/S2_ndvi_max_2022.tif 

Generating URL ...
Please wait ...
Data downloaded to /content/S2_ndvi_max_2022.tif
All the images in the collection have been exported


In [ ]:
myclass.get_export_single(
    image=Landsat.max(),
    name='max_power.tif',
    scale=10,
    crs="EPSG:32629"
)

Exportación de Gif... Donde empezó todo 🥰

In [38]:
myclass.get_gif()

Year 2020: Successfully processed 12 periods using ndvi index
Year 2021: Successfully processed 12 periods using ndvi index
Year 2022: Successfully processed 12 periods using ndvi index
Year 2020: Successfully processed 12 periods using ndvi index
Year 2021: Successfully processed 12 periods using ndvi index
Year 2022: Successfully processed 12 periods using ndvi index
Generating URL...
Please wait ...
The GIF image has been saved to: /content/mygif.gif


### Si queremos usar export a Asset o a Drive tenemos que logearnos de este modo

In [ ]:
import ee, os, pathlib, shutil

# 1) Borra credenciales para forzar reauth
cred_path = pathlib.Path.home() / ".config" / "earthengine" / "credentials"
if cred_path.exists():
    shutil.rmtree(cred_path.parent)  # borra la carpeta earthengine completa

# 2) Autentica con scopes (usa la UI del notebook)
ee.Authenticate(
    auth_mode='notebook',
    scopes=[
        'https://www.googleapis.com/auth/earthengine',
        'https://www.googleapis.com/auth/drive',
    ]
)

# 3) Inicializa
ee.Initialize(project='ee-digdgeografo')

In [ ]:
task = proc.export_to_drive(
    image=winter_p95_alltime,
    description="mndwi_winter_p95_alltime",
    folder="ndvi2gif_exports",
    file_format="GeoTIFF",
    crs="EPSG:32629",
    scale=30,
    max_pixels=int(1e13),
    file_dimensions=8192,    # opcional para trocear
    clip_region=True        # opcional para recortar
    #format_options={"cloudOptimized": True},  # <-- sin 'compression'
)
print("Task:", task.id)

In [ ]:
task = proc.export_to_asset(
    image=winter_p95_alltime,  # MNDWI = float
    asset_id="users/ee-digdgeografo/assets/mndwi_winter_p95_alltime",
    description="mndwi_winter_p95_alltime",
    region=proc.roi,
    crs="EPSG:32630",
    scale=30,
    pyramiding_policy={"MNDWI_winter_p95_alltime": "mean"},  # <- float
    clip_region=True,
    overwrite=True
)